In [ ]:
import math
import rospy
import pickle
import time

from gazebo_msgs.srv import SetModelState
from gazebo_msgs.msg import ModelState
from sensor_msgs.msg import LaserScan

from tf.transformations import quaternion_from_euler

In [ ]:
env_row = 21
env_col = 21
obstacles = [[3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9],
             [8, 9], [8, 10], [8, 11], [8, 12], [8, 13], [8, 14], [8, 15], [8, 16], [8, 17], [8, 18], [8, 19], [8, 20],
             [14, 0], [14, 1], [14, 2], [14, 3], [14, 4], [14, 5], [14, 6], [14, 7], [14, 8], [14, 9]]
laser_data = None

In [ ]:
def laser_callback(msg):

    global laser_data
    laser_data = msg.ranges

In [ ]:
rospy.init_node('get_lidar_data', anonymous=True)
lidar =  rospy.Subscriber("/scan", LaserScan, laser_callback)
set_model_state_proxy = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

In [ ]:
def reset_atom(pos_x, pos_y):
    
    set_model_state_msg = ModelState()
    set_model_state_msg.model_name = 'atom'
    set_model_state_msg.pose.position.x = pos_x
    set_model_state_msg.pose.position.y = pos_y
    set_model_state_msg.pose.position.z = 0
    q_x, q_y, q_z, q_w = quaternion_from_euler(0, 0, math.pi/2)
    set_model_state_msg.pose.orientation.x = q_x
    set_model_state_msg.pose.orientation.y = q_y
    set_model_state_msg.pose.orientation.z = q_z
    set_model_state_msg.pose.orientation.w = q_w
    set_model_state_proxy(set_model_state_msg)

In [ ]:
lidar_data = {}
for i in range(env_row):
    for j in range(env_col):
        if [i, j] in obstacles:
            continue
        reset_atom(i-10, j-10)
        time.sleep(1)
        lidar_data[(i, j)] = laser_data

In [ ]:
filee = open(f'lidar_data_{env_row}x{env_col}.bin', 'wb')
pickle.dump(lidar_data, filee)
filee.close()